In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import sqlite3

In [2]:
co2 = pd.read_csv('CO2_Emissions_1960-2018.csv')
# co2.head()
usa_co2 = co2[co2["Country Name"] == "United States"].transpose()
usa_co2 = usa_co2[1:].astype('float64')
usa_co2 = usa_co2.rename(columns={251: 'AVG CO2 EMISSIONS'})
usa_co2.index = usa_co2.index.astype('int64')
usa_co2.head()

,AVG CO2 EMISSIONS
1960,15.999779
1961,15.681256
1962,16.013937
1963,16.482762
1964,16.968119


In [ ]:
con = sqlite3.connect("FPA_FOD_20170508.sqlite")
# c = con.cursor()
# c.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(c.fetchall())
fires_df = pd.read_sql_query("SELECT * from Fires", con)

In [ ]:
print(list(fires_df.columns))
fires_df.head()

In [ ]:
fire_count_df = fires_df.groupby("FIRE_YEAR")["OBJECTID"].count().to_frame().astype('float64')
fire_size_df = fires_df.groupby("FIRE_YEAR")["FIRE_SIZE"].mean().to_frame().astype('float64')
# print(fire_size_df)
# usa_co2.loc[1992:2015]

In [ ]:
to_analyze = pd.concat([fire_count_df[:2015], usa_co2.loc[1992:2015], fire_size_df], axis=1)
to_analyze.rename(columns={'OBJECTID': 'FIRE COUNT'}, inplace=True)
to_analyze["FIRE COUNT"] /= 1000
# fire count represented in thousands
# co2 is metric tons per capita
# fire size is avg size of all fires in that year
to_analyze

In [ ]:
sns.lineplot(data=to_analyze,x=to_analyze.index,y="AVG CO2 EMISSIONS",label="AVG CO2 EMISSIONS")

In [ ]:
sns.lineplot(data=to_analyze,x=to_analyze.index,y="FIRE COUNT",label="FIRE COUNT")

In [ ]:
sns.relplot(x="AVG CO2 EMISSIONS",y="FIRE COUNT",data=to_analyze,hue='FIRE_SIZE')

In [ ]:
# print(to_analyze['AVG CO2 EMISSIONS'].corr(to_analyze['FIRE COUNT']))

cor_count, p_count = stats.pearsonr(to_analyze['AVG CO2 EMISSIONS'], to_analyze['FIRE COUNT'])
cor_size, p_size = stats.pearsonr(to_analyze['AVG CO2 EMISSIONS'], to_analyze['FIRE_SIZE'])

print(f'Correlation between emissions and fire count: {cor_count}, p = {p_count}')
print(f'Correlation between emissions and fire size: {cor_size}, p = {p_size}')

In [ ]:
east = ["ME","NH","VT","MA","RI","CT","NJ","DE","MD","NY","PA","VA","WV","NC","SC","GA","FL","OH","KY","TN","AL","MS","WI","IL","IN","MI"]
west = ["WA","OR","CA","NV","ID","UT","AZ","MT","AK","HI","WY","CO","NM","TX","ND","SD","NE","KS","OK","LA","AR","MO","IA","MN"]

by_state = fires_df.groupby("STATE")["OBJECTID"].count().to_frame()
east_fires = fires_df[fires_df["STATE"].isin(east)]
west_fires = fires_df[fires_df["STATE"].isin(west)]
print(east_fires.shape[0])
print(west_fires.shape[0])